In [44]:
import tool
import scipy
import pandas as pd
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from ipywidgets import interact, SelectMultiple, fixed
import warnings
warnings.filterwarnings('ignore')

In [45]:
"""
Data initialisation
"""

asset_index = pd.read_csv("data/AIDX.csv", encoding='gbk')

num_limit = (20, 30)
index_list = asset_index['S_IRDCODE'].drop_duplicates().sample(np.random.randint(*num_limit)).tolist()

asset_index['TRADE_DT'] = pd.to_datetime(asset_index['TRADE_DT'], format='%Y%m%d')
asset_index.sort_values(by='TRADE_DT', inplace=True)
asset_index.set_index('TRADE_DT', inplace=True)
asset_index = asset_index.pivot(columns='S_IRDCODE', values='CLOSE').ffill()[index_list].dropna()

print(index_list)
print(asset_index)

['000037.SH', '930634.CSI', '931165.CSI', '931350.CSI', '931226CNY010.CSI', 'h30559.CSI', '930627.CSI', 'n99304.CSI', 'h00068.SH', 'h30547.CSI', '921460.SH', '000111.SH', '399698.SZ', '931135.CSI', '931845CNY01.CSI', '931845.SH', '931985.CSI', 'CN6004.CNI', '932117CNY010.CSI', '931239CNY100.CSI', '931780CNY01.CSI', 'h50029.SH']
S_IRDCODE   000037.SH  930634.CSI  931165.CSI  931350.CSI  931226CNY010.CSI  \
TRADE_DT                                                                      
2023-01-03  7228.4657    734.3433   5825.0322   1718.8475         4876.9426   
2023-01-04  7271.5100    739.6267   5794.0012   1712.7720         4854.7112   
2023-01-05  7434.7903    737.0686   5896.1423   1751.0670         4925.7176   
2023-01-06  7375.7896    730.8736   5852.4387   1746.4723         4968.7322   
2023-01-09  7430.7662    732.9373   5856.2466   1766.2999         5009.7616   
...               ...         ...         ...         ...               ...   
2023-12-20  6332.9390    717.8673   59

In [46]:
"""
Parameters
"""
BACKTEST_DAY = 30
MODEL_TYPE = 'BL' # MVO, RP, BL, RB
TARGET_RETURN = 0.0 # target return
RISK_FREE_RATE = 0.02 # risk-free rate
REBALANCE_DAYS = 30

N = len(index_list)
index_min_weight = [0 for _ in range(N)]
index_max_weight = [1 for _ in range(N)]
WEIGHT_CONSTRAINTS = list(zip(index_min_weight, index_max_weight))

In [47]:
"""
Rebalancing
"""
def rebalance(asset_index, T):
    predicts = []
    actuals = []
    realities = []
    
    for i in range(T, len(asset_index), T):
        
        if i+T >= len(asset_index):
            break
        
        historical_data = asset_index[i-T:i]
        future_data = asset_index[i:i+T]
        
        predict, actual = tool.evaluate(historical_data, future_data, WEIGHT_CONSTRAINTS, MODEL_TYPE, TARGET_RETURN, RISK_FREE_RATE)
        predicts.append(predict)
        actuals.append(actual)
        
        reality = tool.check([1 / N for _ in range(N)], future_data, RISK_FREE_RATE)  # equally weighed
        realities.append(reality)
    
    return predicts, actuals, realities

predicts, actuals, realities = rebalance(asset_index, REBALANCE_DAYS)
print(predicts)
print(actuals)
print(realities)

[(0.06677647520352589, 12.151379614083385, 0.003849478552156514), (-0.21541495935108607, 21.684426056223433, -0.010856407208597617), (0.92053767059139, 61.33328519684508, 0.014682690935291898), (0.15440054245001722, 68.62423077377287, 0.0019584998029789657), (0.019354659526145866, 15.252685280448096, -4.2309958016466414e-05), (-0.025841076154175406, 16.319779461530754, -0.0028089274283535953), (-0.04168234721561069, 14.419555842049164, -0.0042776870446825785)]
[(-0.06305509991142702, 24.45915794301569, -0.0033956647283167572), (0.26326681693115617, 28.210194157128164, 0.00862336556693593), (-0.09513682002402524, 34.30414855133989, -0.003356352653723804), (0.07141097038626977, 90.89970643049618, 0.0005655790585592229), (0.015370960039869232, 24.2897315325426, -0.00019057600344116318), (-0.049489986551150555, 10.267362613636502, -0.006768046397704712), (0.008391106462823667, 14.477625695667387, -0.0008018506474200698)]
[(0.04275472939670015, 54.82441075494544, 0.0004150474046754357), (0.

In [48]:
"""
Output
"""

def display(L1, L2, L3, normalize=False, lines_to_show=None):
    line_names = ['r1', 'v1', 's1', 'r2', 'v2', 's2', 'r3', 'v3', 's3']
    line_styles = {
        'r1': 'r-', 'v1': 'r--', 's1': 'r:',
        'r2': 'b-', 'v2': 'b--', 's2': 'b:',
        'r3': 'g-', 'v3': 'g--', 's3': 'g:'
    }

    a1, b1, c1 = zip(*L1)
    a2, b2, c2 = zip(*L2)
    a3, b3, c3 = zip(*L3)

    lines = {'r1': a1, 'v1': b1, 's1': c1, 
             'r2': a2, 'v2': b2, 's2': c2,
             'r3': a3, 'v3': b3, 's3': c3}

    def normalize_data(data):
        return (data - np.mean(data)) / np.std(data)

    if normalize:
        lines = {name: normalize_data(data) for name, data in lines.items()}

    plt.figure(figsize=(10, 6))
    for line in lines_to_show:
        plt.plot(lines[line], line_styles[line], label=line, marker='o')
    
    plt.title("R: Return\tV: Volatility\tS: Sharpe Ratio\n1: Predicted\t2: Actual\t3: Reality (Equally Weighed)\nButton 'normalize': Normalise each line")
    plt.legend()
    
    for pair in [('r1', 'r2'), ('v1', 'v2'), ('s1', 's2')]:
        if pair[0] in lines_to_show and pair[1] in lines_to_show:
            corr, _ = scipy.stats.spearmanr(lines[pair[0]], lines[pair[1]])
            print(f"Spearman correlation between {pair[0]} and {pair[1]}: {corr:.2f}")
    
    plt.show()


# @interact
interact(display, 
         L1=fixed(predicts), 
         L2=fixed(actuals),
         L3=fixed(realities), 
         normalize=True, 
         lines_to_show=SelectMultiple(options=['r1', 'r2', 'r3', 'v1', 'v2', 'v3', 's1', 's2', 's3'],
                                      value=['r1', 'r2', 'r3', 'v1', 'v2', 'v3', 's1', 's2', 's3'], 
                                      description='Lines'))



interactive(children=(Checkbox(value=True, description='normalize'), SelectMultiple(description='Lines', index…

<function __main__.display(L1, L2, L3, normalize=False, lines_to_show=None)>